# Simple RNN Practice

![alt text](./SimpleRNN.png)

## 1) Import Required Libraries

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np

## 2) Hyperparameter & Data Preprocessing

In [2]:
# Preprocessing string data
# alphabet(0-25), space(26), start(27), end(28) -> 29 chars (0-28)

n_hidden = 35 
lr = 0.01
epochs = 2000

string = "hello pytorch.how long can a rnn cell remember? show us your limit!"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
char_len = len(char_list)
n_letters = len(char_list)

## 3) String to One-hot

In [3]:
# String to onehot vector
# a -> [1 0 0 ... 0 0]

def string_to_onehot(string):
    start = np.zeros(shape=len(char_list) ,dtype=int)
    end = np.zeros(shape=len(char_list) ,dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=char_len ,dtype=int)
        zero[idx]=1
        start = np.vstack([start,zero])
    output = np.vstack([start,end])
    return output

## 4) One-hot to Character

In [4]:
# Onehot vector to word
# [1 0 0 ... 0 0] -> a 

def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

## 5) RNN class

In [5]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.act_fn = nn.Tanh()
    
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.act_fn(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(1, self.hidden_size))
    
rnn = RNN(n_letters, n_hidden, n_letters)

## 6) Loss function & Optimizer

In [7]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

## 7) Train

In [8]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()

    for j in range(one_hot.size()[0]-1):
        input = Variable(one_hot[j:j+1,:])
        output, hidden = rnn.forward(input, hidden)
        target = Variable(one_hot[j+1])
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss
        input = output

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(total_loss)

Variable containing:
 2.7075
[torch.FloatTensor of size 1]

Variable containing:
 1.1269
[torch.FloatTensor of size 1]

Variable containing:
 0.6471
[torch.FloatTensor of size 1]

Variable containing:
 0.3884
[torch.FloatTensor of size 1]

Variable containing:
 0.2649
[torch.FloatTensor of size 1]

Variable containing:
 0.1946
[torch.FloatTensor of size 1]

Variable containing:
 0.1484
[torch.FloatTensor of size 1]

Variable containing:
 0.1161
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  9.2767
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  7.5996
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  6.2679
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  5.3627
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  5.1410
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  4.2967
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  3.8808
[torch.FloatTensor of size

Variable containing:
1.00000e-03 *
  1.3764
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.3177
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.2730
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.2283
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.2144
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  2.5331
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  3.6101
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  2.2004
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.7033
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.3243
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.2168
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.1515
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-03 *
  1.0942
[torch.FloatTensor of size 1]

Variable containing:
1.00

## 8) Test

In [9]:
hidden = rnn.init_hidden()
input = Variable(one_hot[0:1,:])

for i in range(len(string)):
    output, hidden = rnn.forward(input, hidden)
    print(onehot_to_word(output.data))
    input = output

h
e
l
l
o
 
p
y
t
o
r
c
h
.
h
o
w
 
l
 
r
e
o
e
o
l
i
 
o
p
y
p
y
n
u
n
u
h
u
h
r
h
r
e
g
e
g
e
g
l
g
l
o
l
o
l
o
l
o
o
o
o
o
o
o
o
o
